<a href="https://colab.research.google.com/github/kav0001/Capstone-Project-Predicting-Career-Domain-and-Seniority-from-LinkedIn-Profiles/blob/main/PDS_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import unicodedata

def normalize_title(title: str) -> str:
    s = str(title).lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))  # remove accents
    s = re.sub(r"[^\w\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# 1) Director (включая managing director / direktor)
DIRECTOR_PAT = re.compile(
    r"\b(director|direktor|directeur|managing director|vertriebsdirektor|direktion)\b"
)

# 2) Management (C-level / VP / President / Geschäftsführer / Vorstand / Founder / Owner)
MANAGEMENT_PAT = re.compile(
    r"\b(ceo|cto|cfo|coo|cio|cdo|chief|svp|evp|vp|vice president|president|"
    r"owner|founder|co founder|gruender|grunder|"
    r"geschaftsfuhrer|geschaftsfuhrerin|geschaftsfuhrung|vorstand|vorsitzender|partner)\b"
)

# 3) Lead (Head, Leiter*, Leitung*, Teamlead*, Projektleiter* … + составные слова)
HEAD_PAT = re.compile(r"\bhead\b")
LEAD_PAT = re.compile(
    r"\b(\w*leiter\w*|\w*leitung\w*|teamlead|team lead|principal|staff)\b"
)
# (слово "lead" отдельно можно добавить, но оно иногда шумит)
LEAD_WORD_PAT = re.compile(r"\blead\b")

# 4) Junior (явные junior-сигналы + analyst и немецкие junior-роли)
JUNIOR_EXPLICIT_PAT = re.compile(
    r"\b(junior|jr|praktikant|werkstudent|trainee|intern|internship|apprentice|entry|graduate)\b"
)
JUNIOR_ROLE_PAT = re.compile(
    r"\b(mitarbeiterin|mitarbeiter|referentin|referent|assistentin|assistent|analyst)\b"
)

SENIOR_CUES_PAT = re.compile(r"\b(senior|sr|manager|engineer|consultant|specialist|expert)\b")

def predict_seniority(position: str) -> str:
    s = normalize_title(position)

    # 1) Director
    if DIRECTOR_PAT.search(s):
        return "Director"

    # 2) Management
    if MANAGEMENT_PAT.search(s):
        return "Management"

    # 3) Lead
    # "Head of ..." чаще Lead, но "Assistant to Head of ..." лучше не относить к Lead
    if HEAD_PAT.search(s):
        if ("assistant" in s) or re.search(r"\bmanager\b", s):
            pass
        else:
            return "Lead"

    if LEAD_PAT.search(s):
        return "Lead"

    # если хочешь, включи слово "lead" как отдельный сигнал:
    if LEAD_WORD_PAT.search(s):
        return "Lead"

    # 4) Junior
    if JUNIOR_EXPLICIT_PAT.search(s):
        return "Junior"

    if JUNIOR_ROLE_PAT.search(s):
        # если явно senior, то senior
        if re.search(r"\b(senior|sr)\b", s):
            return "Senior"
        return "Junior"

    # 5) Senior (по умолчанию)
    if SENIOR_CUES_PAT.search(s):
        return "Senior"

    return "Senior"

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

df = pd.read_csv("https://drive.google.com/file/d/17rxzgzyZbgtBV0gj9Ai1sBmnrrggShZj/view?usp=sharing")   # столбцы: text, label
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])

test_pred = test_df["text"].map(predict_seniority)
print(classification_report(test_df["label"], test_pred))

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 4699
